In [166]:
import os
import numpy as np
import random
import cv2
import pandas as pd

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import Normalize

In [ ]:
'''
실행시키지 말 것 dataset zip파일 압축 푸는 코드
'''
# import zipfile
# DIV2K = zipfile.ZipFile('/home/lahj91/SR/DIV2K_valid_HR.zip')
# DIV2K.extractall('/home/lahj91/SR/DIV2K')
# DIV2K.close()

In [144]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
# name = os.listdir('./SR_training_datasets/BSDS200')#해당주소에서 data 이름 list로 추출
# df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
# df.to_csv("BSDS200_DataName.csv", index=False) #csv 파일로 저장

In [290]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
# name = os.listdir('./DIV2K/DIV2K_train_HR')
# df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
# df.to_csv("DIV2K_train_DataName.csv", index=False) #csv 파일로 저장

In [299]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
# name = os.listdir('./DIV2K/DIV2K_valid_HR')
# df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
# df.to_csv("DIV2K_valid_DataName.csv", index=False) #csv 파일로 저장

In [192]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
# name = os.listdir('/home/lahj91/SR/SR_testing_datasets/BSDS100')
# df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
# df.to_csv("valid_BSDS200.csv", index=False) #csv 파일로 저장

In [157]:
'''
부가적인 함수들
'''
def img2tensor(img, normalize=True):
    # handle gray image
    if len(img.shape)==2:
        img = img[:,:,np.newaxis]
    img = np.transpose(img, axes=(2,0,1)).astype(np.float32)
    if normalize:
        img = img / img.max()
    return torch.from_numpy(img)

def tensor2img(tensor):
    if len(tensor.shape)==2:
        img = tensor.cpu().numpy()
    elif len(tensor.shape)==3:
        img = np.transpose(tensor.cpu().numpy(), axes=(1,2,0))
    return img

def read_img_as_tensor(img_path, img_size=224, normalize=True):
    if isinstance(img_size, int):
        img_size = (img_size, img_size)
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_size)
    img = img2tensor(img, normalize=normalize)
    return img

In [189]:
'''
dataset loader code
'''
class dataset_SR(Dataset):
    def __init__(self,
                SR_mode = 2,
                flip=0.5, 
                rotation = [cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_180, cv2.ROTATE_90_COUNTERCLOCKWISE],
                degradation = [cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_AREA, cv2.INTER_CUBIC, cv2.INTER_LANCZOS4], 
                channel_wise_noise = True, 
                normalization = True,
                data='DIV2K',
                setting = "train"):
        super(dataset_SR, self).__init__()
        self.SR_mode = SR_mode
        self.flip = flip
        self.rotation = rotation
        self.data = data
        self.degradation = degradation
        self.channel_wise_noise = channel_wise_noise
        self.normalization = normalization
        self.setting = setting
        self.mode_path = {"train" : '/home/lahj91/SR/SR_training_datasets', "val" : '/home/lahj91/SR/SR_testing_datasets'}
        self.data_name_list = pd.read_csv('./DataName/' + self.setting + "_" + self.data + ".csv")
        self.normalize_img = Normalize(mean=[0.406, 0.456, 0.485], std=[0.225, 0.224, 0.229])
    def __len__(self):
        return len(self.data_name_list)
    def __getitem__(self, idx):
        view_path = self.mode_path[self.setting]+ '/'+ self.data + '/' + self.data_name_list.iloc[idx]['name']
        origin = cv2.imread(view_path, cv2.IMREAD_UNCHANGED)
        p, q = random.randint(0, origin.shape[0]-self.SR_mode*48), random.randint(0, origin.shape[1]-self.SR_mode*48)
        origin = origin[p:p+self.SR_mode*48,  q:q+self.SR_mode*48] #random crop(96*96)으로 origin data 생성
        if self.channel_wise_noise: #channel-wise noise
            pn = np.random.uniform(*[1 - 0.4, 1 + 0.4], size=(3))
            origin = np.minimum(255., np.maximum(0., origin * pn[np.newaxis, np.newaxis, :]))
        if random.random() < self.flip: #좌우반전
            origin = cv2.flip(origin, 1)
        if self.rotation != False: #rotation
            rd = random.random()
            if rd < 3/4:
                origin = cv2.rotate(origin, random.choice(self.rotation))
            else:
                origin = origin
        degraded = cv2.resize(origin, dsize=(int(self.SR_mode*24),int(self.SR_mode*24)), fx=0.5, fy=0.5, interpolation=random.choice(self.degradation)) 
        interpolated = cv2.resize(degraded, dsize=(self.SR_mode*48,self.SR_mode*48), fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC) #bicubic으로 degraded를 96*96 size 복원
        return torch.from_numpy(origin.transpose(2,0,1))/255, self.normalize_img(torch.from_numpy(degraded.transpose(2,0,1))/255), self.normalize_img(torch.from_numpy(interpolated.transpose(2,0,1))/255)

In [190]:
dataloader = dataset_SR() #dataset_SR class 불러오기
origin, degraded, interpolated = dataloader.__getitem__(2) #test code, 저장하여 확인(cv2.imshow가 안됨;;; 쓰지말 것 kernel 터짐)
'''
아래는 잘 뽑히고 있는지 시각화(저장됨)하는 코드
'''
cv2.imwrite('/home/lahj91/SR/origin.png',tensor2img(origin))
cv2.imwrite('/home/lahj91/SR/degraded.png',tensor2img(degraded))
cv2.imwrite('/home/lahj91/SR/interpolated.png',tensor2img(interpolated))

True

In [191]:
origin

tensor([[[0.7039, 0.6436, 0.6597,  ..., 0.0965, 0.2172, 0.3459],
         [0.6959, 0.6717, 0.6516,  ..., 0.2092, 0.2092, 0.2936],
         [0.6717, 0.6838, 0.6516,  ..., 0.1730, 0.2373, 0.2413],
         ...,
         [0.6436, 0.5832, 0.3459,  ..., 0.4143, 0.4746, 0.4827],
         [0.5953, 0.6315, 0.4867,  ..., 0.4505, 0.4827, 0.5309],
         [0.5510, 0.6114, 0.5752,  ..., 0.4585, 0.5832, 0.5470]],

        [[1.0000, 1.0000, 1.0000,  ..., 0.1135, 0.2615, 0.4242],
         [1.0000, 1.0000, 1.0000,  ..., 0.2763, 0.2516, 0.3552],
         [1.0000, 1.0000, 1.0000,  ..., 0.2861, 0.3108, 0.2812],
         ...,
         [1.0000, 1.0000, 0.8288,  ..., 0.7844, 0.8436, 0.8485],
         [1.0000, 1.0000, 0.9619,  ..., 0.8140, 0.8288, 0.8830],
         [1.0000, 1.0000, 1.0000,  ..., 0.8090, 0.9373, 0.9126]],

        [[0.6666, 0.6509, 0.6431,  ..., 0.0680, 0.1490, 0.2301],
         [0.6614, 0.6614, 0.6353,  ..., 0.1490, 0.1438, 0.1987],
         [0.6562, 0.6640, 0.6379,  ..., 0.1595, 0.1752, 0.

In [171]:
'''
torch DataLoader 작동 확인
'''
dataloader = dataset_SR() #dataset_SR class 불러오기
dataloader = DataLoader(dataloader, batch_size=3, shuffle=True) #torch datloader 정의